<a href="https://colab.research.google.com/github/jeffheaton/t81_558_deep_learning/blob/master/t81_558_class_11_03_embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# T81-558: Applications of Deep Neural Networks
**Module 11: Natural Language Processing and Speech Recognition**
* Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
* For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).

# Module 11 Material

* Part 11.1: Getting Started with Spacy in Python [[Video]](https://www.youtube.com/watch?v=A5BtU9vXzu8&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_11_01_spacy.ipynb)
* Part 11.2: Word2Vec and Text Classification [[Video]](https://www.youtube.com/watch?v=nWxtRlpObIs&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_11_02_word2vec.ipynb)
* **Part 11.3: What are Embedding Layers in Keras** [[Video]](https://www.youtube.com/watch?v=OuNH5kT-aD0&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_11_03_embedding.ipynb)
* Part 11.4: Natural Language Processing with Spacy and Keras [[Video]](https://www.youtube.com/watch?v=BKgwjhao5DU&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_11_04_text_nlp.ipynb)
* Part 11.5: Learning English from Scratch with Keras and TensorFlow [[Video]](https://www.youtube.com/watch?v=Y1khuuSjZzc&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN&index=58) [[Notebook]](t81_558_class_11_05_english_scratch.ipynb)

# Google CoLab Instructions

The following code ensures that Google CoLab is running the correct version of TensorFlow.

In [1]:
try:
    %tensorflow_version 2.x
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

Note: not using Google CoLab


# Part 11.3: What are Embedding Layers in Keras

[Embedding Layers](https://keras.io/layers/embeddings/) are a powerful feature of Keras that allow additional information to be automatically inserted into your neural network.  In the previous section you saw that Word2Vec can expand words to a 300 dimension vector.  An embedding layer would allow you to automatically insert these 300-dimension vectors in the place of word-indexes.  

Embedding layers are often used with Natural Language Processing (NLP); however, they can be used in any instance where you wish to insert a larger vector in the place of an index value.  In some ways you can think of an embedding layer as dimension expansion. However, the hope is that these additional dimensions will provide more information to the model and provide a better score.

### Simple Embedding Layer Example

* **input_dim** = How large is the vocabulary?  How many categories are you encoding. This is the number of items in your "lookup table".
* **output_dim** = How many numbers in the vector that you wish to return. 
* **input_length** = How many items are in the input feature vector that you need to transform?

Now we create one that has a vocabulary size of 10, will reduce those values between 0-9 to 4 number vectors.  Each feature vector coming in will have 2 such features.  This neural network does nothing more than pass the embedding on to the output.  But it does let us see what the embedding is doing.

In [2]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
import numpy as np

model = Sequential()
embedding_layer = Embedding(input_dim=10, output_dim=4, input_length=2)
model.add(embedding_layer)
model.compile('adam', 'mse')

Now lets query the neural network with 2 rows.

In [3]:
input_data = np.array([
    [1,2]
])

pred = model.predict(input_data)

print(input_data.shape)
print(pred)

(1, 2)
[[[ 0.00045583  0.01162986  0.02153066  0.02177006]
  [ 0.02861592  0.04424032  0.04959855 -0.03211843]]]


In [4]:
embedding_layer.get_weights()

[array([[-0.0221102 ,  0.00226649,  0.00254037,  0.01190782],
        [ 0.00045583,  0.01162986,  0.02153066,  0.02177006],
        [ 0.02861592,  0.04424032,  0.04959855, -0.03211843],
        [ 0.04582461, -0.02545898, -0.00016803,  0.00339152],
        [ 0.03918555, -0.04752984,  0.03711915,  0.01762971],
        [-0.04822759,  0.03276691, -0.04626492, -0.01514248],
        [-0.00224036, -0.04663466,  0.00142751, -0.00048913],
        [-0.02796851, -0.04063223, -0.04552562,  0.03242474],
        [-0.00985355,  0.03479313, -0.00529178,  0.02721054],
        [-0.02948785, -0.01151751, -0.03079265,  0.00206535]],
       dtype=float32)]

### Transferring An Embedding

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
import numpy as np

embedding_lookup = np.array([
    [1,0,0],
    [0,1,0],
    [0,0,1]
])

model = Sequential()
embedding_layer = Embedding(input_dim=3, output_dim=3, input_length=2)
model.add(embedding_layer)
model.compile('adam', 'mse')

embedding_layer.set_weights([embedding_lookup])

In [6]:
input_data = np.array([
    [0,1]
])

pred = model.predict(input_data)

print(input_data.shape)
print(pred)

(1, 2)
[[[1. 0. 0.]
  [0. 1. 0.]]]


### Training an Embedding

In [7]:
from numpy import array
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Embedding, Dense

In [8]:
# Define 10 resturant reviews.
reviews = [
    'Never coming back!',
    'Horrible service',
    'Rude waitress',
    'Cold food.',
    'Horrible food!',
    'Awesome',
    'Awesome service!',
    'Rocks!',
    'poor work',
    'Couldn\'t have done better']

# Define labels (1=negative, 0=positive)
labels = array([1,1,1,1,1,0,0,0,0,0])

In [9]:
VOCAB_SIZE = 50
encoded_reviews = [one_hot(d, VOCAB_SIZE) for d in reviews]
print(f"Encoded reviews: {encoded_reviews}")

Encoded reviews: [[36, 20, 16], [36, 30], [21, 32], [28, 44], [36, 44], [30], [30, 30], [33], [29, 32], [21, 9, 17, 14]]


In [10]:
MAX_LENGTH = 4

padded_reviews = pad_sequences(encoded_reviews, maxlen=MAX_LENGTH, padding='post')
print(padded_reviews)

[[36 20 16  0]
 [36 30  0  0]
 [21 32  0  0]
 [28 44  0  0]
 [36 44  0  0]
 [30  0  0  0]
 [30 30  0  0]
 [33  0  0  0]
 [29 32  0  0]
 [21  9 17 14]]


In [11]:
model = Sequential()
embedding_layer = Embedding(VOCAB_SIZE, 8, input_length=MAX_LENGTH)
model.add(embedding_layer)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])

print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 4, 8)              400       
_________________________________________________________________
flatten (Flatten)            (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 33        
Total params: 433
Trainable params: 433
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
# fit the model
model.fit(padded_reviews, labels, epochs=100, verbose=0)

In [13]:
print(embedding_layer.get_weights()[0].shape)
print(embedding_layer.get_weights())

(50, 8)
[array([[-0.14637835, -0.12984304, -0.15111117, -0.09053998,  0.13021879,
        -0.10539554, -0.05147085,  0.06912396],
       [-0.00579739, -0.00986332,  0.01467195, -0.03045411, -0.02429543,
        -0.03285533, -0.02688962, -0.02814187],
       [ 0.04165565,  0.04553476, -0.0354664 ,  0.00858508,  0.04029242,
        -0.02380581,  0.01589577,  0.03992004],
       [-0.039357  ,  0.01706238,  0.0153041 , -0.00511563, -0.04465302,
         0.0357861 ,  0.01565594, -0.03416499],
       [ 0.01038007, -0.04379574,  0.0257929 ,  0.01036216,  0.01499831,
        -0.01560997, -0.00298757, -0.02088328],
       [ 0.00280363,  0.03223765, -0.04833974,  0.02341447, -0.00116297,
         0.01185446, -0.03544631,  0.03683075],
       [ 0.02292693, -0.02337421, -0.04099866,  0.04001469,  0.04011053,
        -0.03651067,  0.01003332,  0.02863183],
       [ 0.03145957, -0.0270771 ,  0.04718636,  0.04080942,  0.00179552,
        -0.04767065,  0.01851675, -0.02540805],
       [ 0.04629861, -0

In [14]:
loss, accuracy = model.evaluate(padded_reviews, labels, verbose=0)
print(f'Accuracy: {accuracy}')

Accuracy: 1.0
